In [1]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 

In [19]:
### Load .CSV and 
### subset bleaching events (conditional, e.g. exclude NaN)
### Filter only CodeID, site names and country" 
df = pd.read_csv('CBdbV2.csv',encoding='latin-1', low_memory=False)
df = df[['RECORD_ID','COUNTRY','LAT_corrected','LON_corrected','SEVERITY_CODE','DATE','MONTH','YEAR']]
df = pd.DataFrame(df)
#### Convert index into column
#df.reset_index(level=0, inplace=True)
df['ITEM_ID'] = df.index.astype(str).str.zfill(7)   # add '0s' at the beginning of the number and convert as string 
df.rename(columns={'DATE':'DAY'}, inplace = True) #use "implace = True" to rename the existing DataFrame
df = df[pd.to_numeric(df['DAY'], errors='coerce').notnull()]  # Drop rows that do not have as.numeric DATE (DAY)
df['DAY'] = df['DAY'].apply(lambda x: int(float(x)))
df['DAY'] = df['DAY'].apply('{0:0>2}'.format)
df['MONTH'] = df['MONTH'].apply(lambda x: int(float(x)))
df['MONTH'] = df['MONTH'].apply('{0:0>2}'.format)
df['DAY_RANK'] = df['MONTH'].apply(lambda x: 15 if x == '02' else 16)
df['full_date'] = df.apply(lambda x: '%s-%s-%s' % (x['YEAR'], int(float(x['MONTH'])), x['DAY']), axis=1) # create a column in datetime format; %s - String (or any object with a string representation, like numbers), %d - Integers %f - Floating point numbers
df['full_date'] = df['full_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) # use datetime funtion to convert string 'full_date' to datetime  
df['rank_date'] = df.apply(lambda x: '%s-%s-%s' % (x['YEAR'], int(float(x['MONTH'])), x['DAY_RANK']), axis=1) # create a column in datetime format; %s - String (or any object with a string representation, like numbers), %d - Integers %f - Floating point numbers
df['rank_date'] = df['rank_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) # use datetime funtion to convert string 'full_date' to datetime 
df['JD'] = df['full_date'].apply(lambda x: x.timetuple().tm_yday)
### Re-order by column starting by 'ID' and renaming LAT and LON
df = df.reindex(columns=['ITEM_ID','COUNTRY','LAT_corrected','LON_corrected','SEVERITY_CODE','DAY','MONTH','YEAR','full_date','rank_date','JD'])
df.rename(columns={'LAT_corrected': 'lat', 'LON_corrected': 'lon'}, inplace=True) # use "implace = True" to rename the existing DataFrame (rather than creating a copy)
df = df.dropna(axis=0, how='any')  # Drop row with NaN if any

In [20]:
df

,ITEM_ID,COUNTRY,lat,lon,SEVERITY_CODE,DAY,MONTH,YEAR,full_date,rank_date,JD
358,0000358,United States,28.164800,-94.298300,1,11,10,1993,1993-10-11,1993-10-16,284
384,0000384,United States,28.164800,-94.298300,1,14,05,1994,1994-05-14,1994-05-16,134
390,0000390,United States,28.164800,-94.298300,0,25,09,1994,1994-09-25,1994-09-16,268
477,0000477,United States,28.164800,-94.298300,1,14,05,1995,1995-05-14,1995-05-16,134
495,0000495,United States,28.164800,-94.298300,1,08,10,1995,1995-10-08,1995-10-16,281
...,...,...,...,...,...,...,...,...,...,...,...
37768,0037768,Iran,26.661884,55.915603,0,20,12,2017,2017-12-20,2017-12-16,354
37769,0037769,Jamaica,18.099944,-77.999472,0,20,12,2017,2017-12-20,2017-12-16,354
37770,0037770,Australia,-17.290650,146.478083,0,21,12,2017,2017-12-21,2017-12-16,355
37771,0037771,Iran,26.826644,56.319456,0,21,12,2017,2017-12-21,2017-12-16,355


In [4]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        #string = str(ncfilename)
        #filename = string[-16:] 
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat,method='nearest') 
    val_arr = dsloc['analysed_sst']
    return val_arr 

In [5]:
# Extract arrays per row and put them in a list
home = Path(expanduser("~"))
x = []
for row in df.itertuples():
        lat = row.lat
        lon = row.lon
        time = row.rank_date
        path = home / Path("D:/PGonzalez/SST_CRW/coraltemp_monmean/sst_monmean" + str(row.YEAR) + ".nc") 
        array = file_info(path)
        cf = array_sel(array)
        ranking = cf.rank('time')
        rank_max = max(ranking)
        rank_month = ranking.sel(time=time.strftime("%Y-%m-%d")) # ranking from coldest to warmest
        x.append(rank_max)

In [33]:
coor = [item.coords['time'] for item in x]    # get array of values of coordinate 'time' for analysed sst

In [34]:
# create lists of values (actual time) of coordinates  
coor_list = []
for item in coor: 
    cor_val = item.values
    coor_list.append(cor_val)

In [35]:
df_coor = pd.DataFrame(coor_list, columns=['Date_MMM'])
df_coor['Date_MMM'] = pd.to_datetime(df_coor.Date_MMM)
df_coor['Date_MMM'] = df_coor['Date_MMM'].dt.strftime('%Y-%m-%d')
df_coor

,Date_MMM
0,1993-08-16
1,1994-07-16
2,1994-07-16
3,1995-08-16
4,1995-08-16
...,...
20998,2017-07-16
20999,2017-09-16
21000,2017-02-15
21001,2017-07-16


In [12]:
#df_coor.to_csv('df_dateofMMM.csv', index=False)

In [45]:
list_coor = df_coor['Date_MMM'].values.tolist()

In [46]:
df['date_MMM'] = list_coor
df

In [48]:
#

In [ ]:
# # Create a list of arrays
# z = []
# rmonth = []
# rmax = []
# for item in x: 
#     value = item.sel()
#     val = max(value)
#     rank = value.rank('time')
#     rank_max = max(rank)
#     rank_month = rank.sel(time=time.strftime("%Y-%m-%d"))
#     z.append(val)
#     rmonth.append(rank_month)
#     rmax.append(rank_max)   

In [ ]:
# # create a list for sst values # create lists for coordinates (time) 
# sst = [item[0].values for item in z]   # get array of values; analysed sst in this case
# rm = [item[0].values for item in rmonth]  # get the rank of the month according to the warmest month of the climatology
# rmx = [item[0].values for item in rmax] # get the warmest month of the climatology 

# coor = [item[0].coords['time'] for item in z]    # get array of values of coordinate 'time' for analysed sst

In [ ]:
# # create lists of values (actual time) of coordinates  
# coor_list = []
# rm_list = []
# rmx_list = []
# for item in coor: 
#     cor_val = item.values
#     coor_list.append(cor_val)
# for item in rm: 
#     rm_val = item.values
#     rm_list.append(rm_val)
# for item in rmx: 
#     rmx_val = item.values
#     rmx_list.append(rmx_val)    

In [ ]:
# df_sst = pd.DataFrame(sst, columns = ['MMMmax_SST']) 
# df_coor = pd.DataFrame(coor_list, columns=['Date'])
# df_rm = pd.DataFrame(rm, columns=['rank_month_date'])
# df_rmx = pd.DataFrame(rmx, columns=['warmest_month'])
# df_rank = pd.concat([df_sst, df_coor, df_rm, df_rmx ], axis=1)
# #df.to_csv('df_MMMmax_date.csv', index=False)

In [ ]:
# rank_value = [item[0].values for item in x] 
# df_rank = pd.DataFrame(rank_value, columns = ['rank_cold_to_warm'])
# df_rank

In [ ]:
# def asc(item):
#     'This function reverse the ranking to warmest = 1 to coldest = 12'
#     if item == str(1.0):
#         return '12'
#     elif item == str(2.0):
#         return '11'  
#     elif item == str(3.0):
#         return '10'
#     elif item == str(4.0):
#         return '9'  
#     elif item == str(5.0):
#         return '8'  
#     elif item == str(6.0):
#         return '7'  
#     elif item == str(7.0):
#         return '6'  
#     elif item == str(8.0):
#         return '5'  
#     elif item == str(9.0):
#         return '4'  
#     elif item == str(10.0):
#         return '3'  
#     elif item == str(11.0):
#         return '2'  
#     elif item == str(12.0):
#         return '1'  

In [ ]:
# warm = []
# for row in df_rank.itertuples():
#     rank = row.rank_cold_to_warm
#     s = asc(str(rank))
#     warm.append(s)
# df_warm = pd.DataFrame(warm, columns=['rank_warm_to_cold'])
# df_warm_rank = pd.concat([df_rank, df_warm], axis=1)
# df_warm_rank